In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd
import math
#import json

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from rf import *

In [ ]:
'''
PARAMETERS: modify in notebook_params notebook, then run that notebook and this cell to update here
DO NOT modify this cell
'''

%store -r basic_config
#print(" modelling year is (filter_year param): {} (this is first year if season spans two years)".format(basic_config['filter_yr']))
%store -r classification_params

print("Classification_Params: \n" 
      " temp output files are saved to (local_model_dir): {} \n" 
      " shared modelling files are in (main_model_dir): {} \n" 
      " feature_model = {} \n sample_model = {} \n model_name = {} \n"
      " the full sample pt file: {} \n"
      " the full sample dataframe with the feature model applied: {} \n"
      " the subset pt file based on the sample model: {} \n"
      " sample_model_dict: {} \n lc_class = {}"
      .format(classification_params['model_dir'],classification_params['main_model_dir'],classification_params['feature_model'],
              classification_params['sample_model'],classification_params['model_name'],basic_config['ptfile'],
              classification_params['samp_pix_vars'],classification_params['samp_pts'],
              classification_params['feature_mod_dict'],classification_params['lc_mod']))

## define / alter sample pixels to participate in model training

### load in LUT to see class options

In [ ]:
lut=Path('../Class_LUT.csv')
print(pd.read_csv(lut).sort_values('LC_UNQ')[['LC_UNQ','USE_NAME','LC25','LC25_name']])

### start with default models with pixel-only data:

In [ ]:
pix_vars = pd.read_csv(classification_params['samp_pix_vars'])
print('pix_vars has {} features and {} sample points'.format(pix_vars.shape[1]-1, pix_vars.shape[0]))
print(pix_vars.columns.tolist())

In [ ]:
samp_pts = pd.read_csv(basic_config['ptfile'])
pix_vars = pd.read_csv(classification_params['samp_pix_vars'])
#print(samp_pts)

pix_data = pix_vars.merge(samp_pts, left_on='OID_', right_on='OID_', how='left')

pix_data.drop(['LC2'], axis=1, inplace=True)
pixdf = pix_data.merge(pd.read_csv(lut), left_on='Class', right_on='USE_NAME', how='left')
print('sample breakdown by LC25 class:')
print(pixdf['LC25_name'].value_counts())

print('default rf model with all sample pixels and pixel only data')

## Add smallholder flag to dataset

In [ ]:
## <=1 hectare
pixdf['smlhld_1ha'] = pixdf.apply(lambda x: 1 if (
    ((x['var_poly_pred_area'] < 100) and (x['LC2'] == 1)) or (
    (x['FieldWidth'] <= 100) and (x['LC2'] == 1)) or x['LC25'] == 'Crops-mix') else 0, axis=1)
print(pixdf['smlhld_1ha'].value_counts())
## <= .5 hectare
pixdf['smlhld_halfha'] = pixdf.apply(lambda x: 1 if (
    ((x['var_poly_pred_area'] < 50) and (x['LC2'] == 1)) or (
    (x['FieldWidth'] <= 50) and (x['LC2'] == 1)) or x['LC25'] == 'Crops-mix') else 0, axis=1)
print(pixdf['smlhld_halfha'].value_counts())
#pd.DataFrame.to_csv(pixdf, os.path.join([classification_params['model_dir']/ptsgb_withSmalls.csv')

## Drop classes that have sample sizes too small to model

In [ ]:
dropClass = ['Crops-Vineyard','NewPlant']
pixdf = pixdf[~pixdf['LC25_name'].isin(dropClass)]
#pixdf = pixdf.drop("Description", axis=1)
pd.options.display.max_columns = None
print(pixdf['LC25_name'].value_counts())
#print(pd.DataFrame(pixdf.isnull().any()))

#### check for nan columns if desired

In [ ]:
## Note if any NaN columns start with 'var_', the NaNs will cause the rf model to fail
nancols = pixdf.columns[pixdf.isna().any()].tolist()
print(f'columns with NaN: {nancols}')

#for i in nancols:
#    print(pixdf[i])

## prep pixel datasets by reducing sample

### Option1: by sampling method (reducing dominant CAN soy pts that are not verified in GE)

In [ ]:
#### Note that there are many fewer ground verified soy points -- we want to keep all of these in the sample
soyground = pixdf[(pixdf['LC25_name'] == 'Crops-Soybeans') & (pixdf['SampMethod'] != 'CAN - unverified in GE')]
print(soyground.shape[0])

In [ ]:
#### model_name = pixdf_25Soy
##   removes 3/4 of the soy points because they are far overrepresented
pixdf_25Soy = pixdf[(pixdf['rand']>.8) | (pixdf['SampMethod'] != 'CAN - unverified in GE')]
print('25Soy sample breakdown by LC25 class:')
print(pixdf_25Soy['LC25_name'].value_counts())
pixdf_path = os.path.join(classification_params['model_dir'],'pixdf_25Soy.csv')
pd.DataFrame.to_csv(pixdf_25Soy, pixdf_path)

### Option2: by balancing classes

##### First run this to make sure ground sample points are used first for soy (because sample is overwhelmed by unverified CAN pts)

In [ ]:
#### model name = pixdf_
## Note we need about 1374 soy points for a balanced model. we want to include all the 356 ground points found above + 
allsoy = pixdf['LC25_name'].value_counts()['Crops-Soybeans']
othersoy = (1550 - 356) / allsoy
print(othersoy)
pixdf_balsoy = pixdf[(pixdf['rand'] < othersoy) | (pixdf['SampMethod'] != 'CAN - unverified in GE')]
print('balsoy sample breakdown by LC25 class:')
print(pixdf_balsoy['LC25_name'].value_counts())
#pixdf_path = os.path.join(classification_params['model_dir'],'pixdf_balsoy.csv')

### Option2: by balancing classes

In [ ]:
 # run balance_training_data function
 #   balances class samples based on map proportion, relative to sample size for class with max map proportion
 #   (this estimated map proportion is a column named "perLC25E" in the LUT )
 #   allows a minimum threshold to be set {cutoff} so that sample sizes are not reduced below the minimum
 #   allows a factor to be set for mixed (heterogeneous) classes to sample them more heavily than main classes
 #       (the maximum value will depend on the available samples for these classes. Current max is ~4)
 #   prints 'pixdf_bal{cutoff}mix{mix_factor}.csv' in out_dir

out_dir = classification_params['local_dir']
#out_dir = classification_params['main_model_dir']
balance_training_data(lut, pixdf_balsoy, out_dir, "rebal", cutoff = 500, mix_factor = 8)
## repeat with mix_factor = 2 - 10

In [ ]:
##fix this
new_feature_mod = 'Max_nomon5-6'
classification_params['samp_pix_vars'] = '/home/downspout-cel/paraguay_lc/vector/tests/ptsgdb_{}.csv'.format(new_feature_mod)

In [ ]:
# use this to fix any mess ups u make 
"""
dir = "/home/downspout-cel/paraguay_lc/classification/RF"
for filename in os.listdir(dir):
    if filename.startswith("pixdf_{}".format(new_feature_mod)):
        filepath = os.path.join(dir, filename)
        os.remove(filepath)
"""

In [ ]:
## This is an old method used for the original bal1000 model. Better to use the balance_training_data function

#### sample model_name = bal1000
pixdf1 = pixdf[(pixdf['PurePixel'] != 'No') | (pixdf['LC25_name'].str.contains('mix', na=False, case=False))]
pixdf2 = pixdf1[(pixdf['rand']>.84) | (pixdf['SampMethod'] != 'CAN - unverified in GE')]
pixdf3 = pixdf2[(pixdf2['rand']>.65) | (pixdf2['LC25_name'] != 'Mixed-VegEdge')]
pixdf4 = pixdf3[(pixdf3['rand']>.65) | (pixdf3['LC25_name'] != 'Crops-mix')]
pixdf5 = pixdf4[(pixdf4['rand']>.86) | (pixdf4['LC25_name'] != 'Mixed-path')]
pixdf6 = pixdf5[(pixdf5['rand']>.30) | (pixdf5['LC25_name'] != 'Crops-Yerba-Mate')]
pixdf7 = pixdf6[(pixdf6['rand']>.39) | (pixdf6['SampMethod'] == 'GE_KW_sup') | (pixdf6['LC25_name'] != 'Grassland-Managed')]
#pixdf8 = pixdf7[(pixdf7['rand']>.36) | (pixdf7['LC25_name'] != 'Trees-Forest')]
print('pixdf_bal0 sample breakdown by LC25 class:')
print(pixdf7['LC25_name'].value_counts())
pixdf_path = os.path.join(classification_params['main_model_dir'],'pixdf_base1000.csv')
pd.DataFrame.to_csv(pixdf7, pixdf_path)

## if polygons are available, can combine pixel and polygon dfs and create rf datasets for points with polygons and those without:

In [ ]:
poly_data = pd.read_csv(classification_params['samp_poly'])
#rename column names that also occur in pixel df
poly_data.rename(columns={'area':'areaSeg'}, inplace=True)
all_data = pixdf.merge(poly_data, left_on='OID_', right_on='OID_', how='left')
polypixdf_path = os.path.join(classification_params['model_dir'],'pts_polyData_joinCheck.csv')
pd.DataFrame.to_csv(all_data, polypixdf_path, sep=',', na_rep='NaN', index=True)

### first create dataset for points outside of polygons (here we have no variables to add to the original model)

In [ ]:
outsideSeg = all_data[all_data['areaSeg'].isna()]
print(f'of the {all_data.shape[0]} sample points in our dataset, {outsideSeg.shape[0]} are outside of our segmented polygons')
print(outsideSeg['LC17_name'].value_counts())
out_dir = os.path.join(classification_params['model_dir'],'outside')
rfout17 = rf_model(outsideSeg,out_dir,'All','Permutation',29,'Fullsamp')

And for model with more balanced soy representation (25Soy):

In [ ]:
all_data_25Soy = pixdf_25Soy.merge(poly_data, left_on='OID_', right_on='OID_', how='left')
outsideSeg_25Soy = all_data_25Soy[all_data_25Soy['areaSeg'].isna()]
print(outsideSeg_25Soy['LC17_name'].value_counts())
out_dir = os.path.join(classification_params['model_dir'],'outside')
rfout17_lessSoy = rf_model(outsideSeg_lessSoy,out_dir,'All','Permutation',29,'LessSoy')

#### now create dataset for points inside of polygons (here we want to add some variables first)

In [ ]:
#poly_data['AvgU'] = poly_data.apply(lambda x:count([x[c] for c in df.columns if c.endswith('U')]),axis=1)
#TODO: calculate these in pandas as above
poly_data.rename(columns={'areaSeg':'var_areaSeg','AVGU':'var_AVGU','AVGR':'var_AVGR','AVGSTD':'var_AVGSTD','MAXR':'var_MAXR','MINR':'var_MINR','STDU':'var_STDU','MINU':'var_MINU','MAXU':'var_MAXU','rU':'var_RU'}, inplace=True)
polyvars = pixdf.merge(poly_data, left_on='OID_', right_on='OID_', how='left')
withinSeg = polyvars[polyvars['var_areaSeg'] > 0]

print(withinSeg['LC17_name'].value_counts())
out_dir = os.path.join(classification_params['model_dir'],'within')
#rfin17 = rf_model...

In [ ]:
all_data_lessSoy = pixdf_lessSoy.merge(poly_data, left_on='OID_', right_on='OID_', how='left')
withinSeg_lessSoy = all_data_lessSoy[all_data_lessSoy['var_areaSeg']>0]
print(withinSeg_lessSoy['LC17_name'].value_counts())
out_dir = os.path.join(classification_params['model_dir'],'within')
#rfin17_lessSoy = rf_model(...

## To save an html copy of this notebook with all outputs:
(these two cells should be last in notebook)

In [ ]:
%%script echo skipping
### comment out above line and run this cell to print output as html

out_name = str(basic_config['country']+'6a_RandomFoest_VariableDataframe'+'_model'+str(classification_params['model_name'])+'_'+'Tests1')
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --ExecutePreprocessor.store_widget_state=True --output=$out_name 6b_RandomFoest_ModelComparisons.ipynb